In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os
import json
from urlparse import urlparse
from urllib import urlretrieve, unquote
import datetime
import requests
import sys

<b>Читаем данные</b>

In [4]:
df = pd.read_csv('data/gender_age_dataset.txt', sep='\t')


In [5]:
df.describe()

,gender,age,uid,user_json
count,41138,41138,41138,41138
unique,3,6,41138,41138
top,M,25-34,0dad83b2-bd85-4939-989d-27d0bce2d194,"{""visits"": [{""url"": ""http://www.1001eda.com/sa..."
freq,18698,15457,1,1


In [7]:
#let's parse from json to list
df['user_json'] = df['user_json'].apply(json.loads)

In [8]:
# function that convert url  user_json to domains
def url2domain(urllist):
    result = []
    for l in urllist['visits']: 
        url = l[u'url']
        url = re.sub('(http(s)*://)+', 'http://', url)
        parsed_url = urlparse(unquote(url.strip()))
        if parsed_url.scheme in ['http','https']: 
            netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
            if netloc is not None: 
                result.append(str(netloc.encode('utf8')).strip())
    return result


In [9]:
#adding a new column with parsed sites
df['domain']=df['user_json'].apply(url2domain)

In [10]:
df.head()

,gender,age,uid,user_json,domain
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,{u'visits': [{u'url': u'http://zebra-zoya.ru/2...,"[zebra-zoya.ru, news.yandex.ru, sotovik.ru, ne..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,{u'visits': [{u'url': u'http://sweetrading.ru/...,"[sweetrading.ru, sweetrading.ru, sweetrading.r..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,{u'visits': [{u'url': u'http://ru.oriflame.com...,"[ru.oriflame.com, ru.oriflame.com, ru.oriflame..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,{u'visits': [{u'url': u'http://translate-tatto...,"[translate-tattoo.ru, nadietah.ru, 1obl.ru, 1o..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,{u'visits': [{u'url': u'https://mail.rambler.r...,"[mail.rambler.ru, news.rambler.ru, mail.ramble..."


<b>Прочитаем словарь "домен-категории яндекса", полученный с помощью обработки getting_cat.py </b>

In [12]:
Slovar_cat1 = pd.read_csv('pr1_slovar_full/part-00000', sep='\t', names = ["site", "cat"])
Slovar_cat2 = pd.read_csv('pr1_slovar_full/part-00001', sep='\t', names = ["site", "cat"])
Slovar_cat = Slovar_cat1.append(Slovar_cat2)
Slovar_cat

,site,cat
0,lamoda.kz,"Дом;Покупки;Одежда, обувь, аксессуары;Универса..."
1,regent-decor.ru,Бизнес;Строительство;Стройматериалы;Прочее
2,pirogovclinic.ru,Дом;Здоровье;Специализированная медпомощь
3,elabuga.com,Культура;Музеи;Универсальное
4,peopleandcountries.com,Общество;Прочее
5,butterfly-hotel.ru,Отдых;Туризм;Гостиницы
6,promvest.info,СМИ;Периодика;Журналы
7,gorod99.com,Дом;Покупки;Бытовые услуги;Прочее
8,forum-volgograd.ru,Универсальное
9,bookclub.ua,Культура;Литература;Книжные магазины


<b>Прочитаем словарь "домен-title", полученный с помощью обработки getting_title.py </b>

In [19]:
Slovar_title = pd.read_csv('TITLES.txt', sep='\t', names = ["site", "info"])
Slovar_title

,site,info
0,imperiapallet.ru,поддон паллет деревянный купить выгодный цена...
1,topstatus.ucoz.ru,статус рунет главный страница
2,cc.bingj.com,bing
3,future-lancer.ru,mitsubishi lancer новость технический характе...
4,moms-crochet.com,moms crochet
5,oootdl.narod.ru,предлагать поставка машина оборудование товар...
6,radiochast.ru,интернет магазин электронный компонент радиоч...
7,oazistort.ru,торт заказ спб доставка изготовление торт сан...
8,soccerlivehd.com,футбол трансляция матч качество soccerlivehd
9,ngzt.ru,газета свежий новость екатеринбург свердловск...


<b> Каждому встреченному домену поставим в соотвествие все связанные с ним слова (категории и сайты) </b>

In [ ]:
def all_words(domains):
    result = ""
    flag = 0
    for dom in domains:
        result = result + ';' + str(dom)
        if not Slovar_cat[Slovar_cat['site'] == dom].empty:
            for token in Slovar_cat[Slovar_cat['site'] == dom].iloc[0]['cat'].split(';'):
                result = result + ';' + token.strip().lower()
        Slice = Slovar_title[Slovar_title['site'] == dom]
        if not Slice.empty:
            result = result + ';'+ str(Slice.iloc[0]["info"])
    return result

In [ ]:
df['all_words']=df['domain'].apply(all_words)

In [ ]:
out = df[df.columns.difference(['user_json'])]
out.to_csv('data_project_res_cat_lemm_povt_ok_all.csv', sep='|', encoding='utf-8')